In [13]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
# import SVD
# import accuracy

import warnings; warnings.simplefilter('ignore')

In [12]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [3]:
import json
import pandas as pd
import numpy as np

import requests

from tqdm import tqdm_notebook
from collections import Counter

# 어린이 - 책 소개 기반 추천

- 데이터 불러오기

In [4]:
child_final_df=pd.read_excel('./data/final/content_re/child_final_df.xlsx')

In [5]:
child_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                978 non-null    int64  
 1   ranking           978 non-null    int64  
 2   bookname          978 non-null    object 
 3   authors           978 non-null    object 
 4   publisher         978 non-null    object 
 5   publication_year  480 non-null    float64
 6   isbn13            978 non-null    int64  
 7   addition_symbol   932 non-null    float64
 8   vol               797 non-null    float64
 9   class_no          931 non-null    float64
 10  loan_count_x      978 non-null    int64  
 11  bookImageURL_x    977 non-null    object 
 12  keyword           512 non-null    object 
 13  describe          976 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 107.1+ KB


In [6]:
# 책소개가 없는 데이터지우기
child_final_df=child_final_df[pd.notnull(child_final_df['describe'])]

In [7]:
# 책 제목 중복 제거 
child_final_df.drop_duplicates('bookname',keep='first',inplace=True)

In [8]:
# 인덱스 정리
child_final_df.reset_index(drop=True,inplace=True)

In [9]:
# 제목 양쪽 공백제거
for i in range(len(child_final_df)):
    child_final_df['bookname'][i]=str(child_final_df['bookname'][i]).strip()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
child_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                510 non-null    int64  
 1   ranking           510 non-null    int64  
 2   bookname          510 non-null    object 
 3   authors           510 non-null    object 
 4   publisher         510 non-null    object 
 5   publication_year  442 non-null    float64
 6   isbn13            510 non-null    int64  
 7   addition_symbol   464 non-null    float64
 8   vol               332 non-null    float64
 9   class_no          463 non-null    float64
 10  loan_count_x      510 non-null    int64  
 11  bookImageURL_x    510 non-null    object 
 12  keyword           439 non-null    object 
 13  describe          510 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 55.9+ KB


- 책소개를 형용사+명사 데이터로 바꿈

In [14]:
book_intro=[]
for i in range(len(child_final_df)):
    sentences_tag = []
    lines=child_final_df['describe'][i].replace('\n',' ').split('.')
    twitter = Twitter()
    for sentence in lines:
        morph = twitter.pos(sentence)
        sentences_tag.append(morph)
    noun_adj_list = []
    for sentence1 in sentences_tag:
        for word, tag in sentence1:
            if tag in ['Noun','Adjective']:
                noun_adj_list.append(word)
    book_intro.append(noun_adj_list)

In [15]:
child_final_df['book_intro']=book_intro

In [16]:
intro=[]
for i in range(len(child_final_df)):
    s=' '.join(child_final_df['book_intro'][i])
    intro.append(s)

In [17]:
child_final_df['intro']=intro

In [18]:
child_final_df['describe'].head(3)

0    안전상식 학습만화 「쿠키런 서바이벌 대작전」 제4권 《산과 하늘 편》. 웨어울프맛 ...
1    추리로 배우는 교과서 과학\n\n『어린이 과학 형사대 CSI』시리즈 제5권《CSI,...
2    어린이용 그리스·로마 신화 베스트셀러. 아름다운 여신과 뛰어난 영웅, 그리고 신비로...
Name: describe, dtype: object

In [19]:
child_final_df['intro'].head(3)

0    안전 상식 학습만화 쿠키런 서바이벌 작전 제 권 산 하늘 편 웨어울프 맛 쿠키 도움...
1    추리 교과서 과학 어린이 과학 형사 시리즈 제 권 새로운 위기 처 이 시리즈 어린이...
2    어린이 용 그리스 로마 신화 베스트셀러 아름다운 여신 뛰어난 영웅 신비로운 일 만화...
Name: intro, dtype: object

In [50]:
# 최종데이터 저장
child_final_df.to_csv('./data/final/content_re/child_final.csv',encoding='utf-8-sig',index=False)

# TF-IDF

In [21]:
# 블용어 txt파일 리스트 가져오기
sw=pd.read_csv('./data/final/한국어불용어100.txt', sep = "\t",  engine='python', encoding = "utf-8",header=None)

In [22]:
stop_words=list(sw[0].dropna())

In [41]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(child_final_df['intro'])

In [42]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [43]:
titles = child_final_df['bookname']
indices = pd.Series(child_final_df.index, index=child_final_df['bookname'])

In [44]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [47]:
get_recommendations('쿠키런 서바이벌 대작전').head(10)

201                  쿠키런 서바이벌 대작전 21 (별똥별 산 편)
217        쿠키런 서바이벌 대작전 15 : 무인도 편 - 안전상식 학습만화
245          쿠키런 서바이벌 대작전 5 : 화재 편 - 안전상식 학습만화
251          쿠키런 서바이벌 대작전 6 : 동굴 편 - 안전상식 학습만화
156               쿠키런 서바이벌 대작전 26 : 사이보그의 역습 편
158                  쿠키런 서바이벌 대작전 23 : 차원의 문 편
184                쿠키런 서바이벌 대작전 24 : 쿠키볼의 비밀 편
198                 쿠키런 서바이벌 대작전 27 : 로봇의 심장 편
432    쿠키런 서바이벌 대작전 28 : 최후의 생존자 편 - 안전상식 학습만화
209        쿠키런 서바이벌 대작전 16 : 방사능 편 - 안전상식 학습만화
Name: bookname, dtype: object

In [48]:
get_recommendations('나, 생일 바꿀래!').head(10)

3                                 이사도라 문
91                           마법의 설탕 두 조각
426    빈대 가족의 가난 탈출기 - 짠돌이들에게 배우는 경제 지혜!
38                       (기괴하고 요상한) 귀신딱지
88                           내 멋대로 아빠 뽑기
225                 빈대 가족의 덜렁이는 미운 우리 새끼
259                              할머니는 도둑
326                       엄마 사용법 :김성진 동화
353                            엄마가 사라진 날
246                          엄마는 게임 수업 중
Name: bookname, dtype: object

In [49]:
get_recommendations('초딩의 품격').head(10)

368                초등학생을 위한 나의 라임 오렌지나무 (초등용)
79                                 아홉 살 마음 사전
407                             (책 읽는 강아지) 몽몽
233                                     시간 가게
66                                    스무고개 탐정
445                      퀴즈! 과학상식 : 사물 인터넷 과학
191    마인드 스쿨 15 : 너만 보면 떨려! - 사랑을 푹~ 알게 되는 책
424                                  Why? 해부학
127                       시간을 파는 상점 :김선영 장편소설
303                                Why? 동물 의학
Name: bookname, dtype: object